Ad-hoc computations with Futures
------------------------------------

While many parallel applications can be described as maps, some can be more complex.
In this section we look at the asynchronous Future interface,
which provides a simple API for ad-hoc parallelism.

Futures have additionally characteristics that make them distinct from map:
* They are **Asynchronous**, they start *in the background* according to the scheduler
* Futures can depend on other futures, without requiring the results to be ready
* Future results are not automatically transferred to the client, one must call **result**

This is useful for when your computations don't fit a **regular pattern.**

### Objectives

*  Use the `concurrent.futures` function `submit` to perform ad-hoc parallel computing

### Requirements

*  Pandas
*  concurrent.futures (standard in Python 3, `pip install futures` in Python 2)


    pip install snakeviz
    pip install futures

## Executor.submit

The `submit` method starts a computation in a separate thread or process and immediately gives us a `Future` object that refers to the result.  At first, the future is pending.  Once the function completes the future is finished. 

We collect the result of the task with the `.result()` method,
which does not return until the results are available.

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from time import sleep
e = ThreadPoolExecutor(4)

def slowadd(a, b, delay=1):
    sleep(delay)
    return a + b

In [ ]:
future = e.submit(slowadd, 1, 2, delay=10)
future

In [ ]:
# Keep checking future f (use ctrl-Enter), and at some point state will be finished
future

In [ ]:
future.result()

### Submit many tasks, receive many futures

Because submit returns immediately we can submit many tasks all at once and they will execute in parallel.

In [52]:
%%time
results = [slowadd(i, i) for i in range(10)]

CPU times: user 1.22 ms, sys: 1.75 ms, total: 2.97 ms
Wall time: 10 s


In [53]:
%%time
futures = [e.submit(slowadd, 1, 1) for i in range(10)]
results = [f.result() for f in futures]

CPU times: user 2.73 ms, sys: 3.93 ms, total: 6.66 ms
Wall time: 3.02 s


### Submit different tasks

The virtue of submit is that you can submit different functions and you can perform a bit of logic on each input.

### Exercise: parallelize the following code with e.submit

1.  Replace the `results` list with a list called `futures`
2.  Replace calls to `slowadd` and `slowinc` with `e.submit` calls on those functions
3.  At the end, block on the computation by recreating the `results` list by calling `.result()` on each future in the `futures` list.

In [ ]:
%%time

### Sequential Version

def slowadd(a, b, delay=1):
    sleep(delay)
    return a + b

def slowsub(a, b, delay=1):
    sleep(delay)
    return a - b

results = []
for i in range(5):
    for j in range(5):
        if i < j:
            results.append(slowadd(i, j, delay=1))
        elif i > j:
            results.append(slowsub(i, j, delay=1))

In [ ]:
%%time

### Parallel Version

# TODO

In [ ]:
%load solutions/submit-1.py

### Conclusion on submit

*  Submit fires off a single function call in the background, returning a future.  
*  When we combine submit with a single for loop we recover the functionality of map.  
*  When we want to collect our results we replace each of our futures, `f`, with a call to `f.result()`
*  We can combine submit with multiple for loops and other general programming to get something more general than map.


---

Further practicing
=============

### Exercise: Parallelize pair-wise correlations with `e.submit`


In [ ]:
%%time

### Sequential Code

results = {}

for a in filenames:
    for b in filenames:
        if a != b:
            results[a, b] = series[a].corr(series[b])

In [ ]:
%%time

### Parallel Code

futures = ... # TODO

# TODO

results = ... # TODO

In [ ]:
%load solutions/submit-2.py

### Exercise: Threads vs Processes

Try the exercise above using Processes vs Threads by replacing `e` with a ProcessPoolExecutor:

#### Before

```python
from concurrent.futures import ThreadPoolExecutor
e = ThreadPoolExecutor(4)
```

#### After

```python
from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor(4)
```

How does performance vary?  We'll talk more about the tradeoffs between threads and processes later on in the tutorial.

### Exercise: Break Python by loading the data in parallel

The HDF5 library we use to load our data is not threadsafe and can cause our entire Python session to crash.

In [ ]:
%%time
from concurrent.futures import ThreadPoolExecutor
e = ThreadPoolExecutor(4)

dfs = e.map(pd.read_hdf, filenames)
series = e.map(lambda df: df['x'], dfs)
series = dict(zip(filenames, series))

Conclusion
-----------

*  We learned how `e.submit` can help us to parallelize more complex applications
*  We used `e.submit` to compute pairwise collelations in parallel
*  We learned that this didn't actually speed up our code very much
*  We compared threads against processes to see some performance differences
*  We crashed our Python session by using threads with unsafe code, warning us that parallelism is sometimes dangerous